# AGDC Version 2 Requirements till 30 June 2016 10/12/2015

## Summary

A great deal of work has been done by AGDC partners to outline the key requirements and technical approach for the next version of the AGDC (Version 2). For example, the previous Technical Working Group held a workshop that resulted in a draft Preliminary Design Report that has subsequently been translated to the AGDC wiki ( http://datacube.org.au ). Similarly, prototype functionality has been built to test and verify key concepts for a range of components including the netCDF-4 based Storage Units that are managed under NCIfs NERDIP data management, publishing and general access to data protocols; a Generalised Data Framework (GDF) to access the multidimensional Storage Units; and the AGDC Analytic Engine adding support for interactive Exploratory Data Analysis (EDA).

The following requirements for Version 2 have been mainly based off this thinking and documentation. It is important to note that these requirements are for the current development
effort running through till June 30, 2016. They are designed to guide and define the next step in the development of the AGDC, not as the final destination.


Approved Requirements for AGDC Version 2

These requirements were formally approved by the AGDC Programme Board at its meeting on 10
December, 2015.

Patterns of Use:
AGDC Version 2 will support the following patterns of use.

## 1. Routine national scale product generation. Specifically, Version 2 will include national collections of:
    
 Water Observations from Space.

 Intertidal Characterisation.

 Landsat Fractional Cover.

 Pixel Quality.

 NDVI.

Landsat Surface reflectance statistical summaries:

    Seasonal medians; and
    
    Most-up-to-date observation.
    
In some cases, these collections may be virtual, i.e. they are not pre-computed but rather
computed as they are needed.


## 2. A user should be able to interact with these collections through a web browser including:#

 Clicking on a pixel and displaying a time series (i.e., pixel drill).



### Spatio-temporal (statistical) summaries that would allow users to easily answer questions such as:
    
    How frequently was water observed over catchment y during time period x; and

    What was the surface reflectance for area x at time y?

## 3. Earth Observation (EO) scientists and allied domain specialists will be able to undertake exploratory data analysis. In general this would mean a user will be able to easily retrieve, investigate, visualise, develop algorithms, test, iterate, visualise results and interpret them in the context of other spatio-temporal datasets.

 A key demonstration of this capability will be the availability of functions and data
structures to enable Landsat/MODIS blending.

### Input data
The AGDC will use the following data collections:
 Landsat: TM, ETM+ and OLI/TIRS.
 MODIS: Collection 6 MOD09 (granule) and MOD43 (sinusoidal tiles) that will provide variables
necessary for the Landsat-MODIS blending algorithm.

In [1]:
import datacube.api
dc = datacube.api.API()
products = dc.list_products()
for product in products:
    print product['description']


Landsat 5 NBAR 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)
Landsat 7 NBAR 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)
Landsat 7 Pixel Quality 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)
Landsat 5 NBART 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)
Landsat 5 PQ 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)
Landsat 8 PQ 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)
Landsat 7 NBART 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)
Landsat 8 NBAR 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)
Landsat 8 NBART 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)


It will also include the SRTM 3 second DSM and 1 and 3 second DEMs.

In [2]:
dc.list_storage_type_names()

[u'ls5_nbar_albers',
 u'ls7_nbar_albers',
 u'ls7_pq_albers',
 u'ls5_nbart_albers',
 u'ls5_pq_albers',
 u'ls8_pq_albers',
 u'ls7_nbart_albers',
 u'ls8_nbar_albers',
 u'ls8_nbart_albers']

At a minimum, the Australian implementation of the AGDC will cover all of continental Australia plus a
one tile buffer and the Great Barrier Reef. However, the Boardfs preference would be for the Version
2 to also cover all Commonwealth Marine Reserves.

In [65]:
import folium
from IPython.display import display
import geopandas
from shapely.geometry import mapping
%matplotlib inline

In [4]:
cells = dc.list_cells()

In [104]:
shapes = []
from shapely.geometry import Polygon
for cell in cells:
    llxy = [i * 100000 for i in cell]
    lrxy = (llxy[0] + 100000, llxy[1])
    ulxy = (llxy[0], llxy[1] + 100000)
    urxy = (lrxy[0], ulxy[1])
    shapes.append(Polygon([(ulxy), (urxy), (lrxy), (llxy)]))

In [108]:
gdf = geopandas.GeoDataFrame(crs={'init': 'epsg:3577', 'no_defs': True}, geometry=shapes)
gdf_project = gdf.to_crs(epsg=4326)
geojson = gdf_project.to_json()
bounds = gdf_project.total_bounds
shp = mapping(Polygon([(bounds[0], bounds[-1]),
                       (bounds[2], bounds[-1]),
                       (bounds[2], bounds[1]),
                       (bounds[0], bounds[1])]))

In [110]:
style_function = lambda x: {'fillColor': '#0000ff' if x['type'] == 'Polygon' else '#00ff00'}
mapa = folium.Map(location=[-30,150], zoom_start=4)
polygons = folium.features.GeoJson(geojson)
poly = folium.features.GeoJson(shp, style_function=style_function)
mapa.add_children(polygons)
mapa.add_children(poly)
display(mapa)

All data collections that are included in the Australian implementation of the AGDC will:
 Have a CC BY Attribution 3.0 or CC BY Attribution 4.0 license. 

The use of a collection with a
different licence will require approval of the AGDC Programme Board; and
 Be in netCDF 4 format and will comply with relevant CF conventions;.

## Output Products
By June 30, 2016 the Australian implementation of the AGDC will ensure that the products being
produced are supported by and hosted on the RDS.

## Technical Requirements
In line with the use-case patterns outlined above, Version 2 will support:
 Data-fusion and analysis across heterogeneous gridded data collections from different
domains.

Tuneable configuration, at ingest, of multidimensional files (eg. chunking, compression type,
dimension depth).

## A data retrieval mechanism that provides the ability to:


Obtain seamless subsets of data across storage unit boundaries;



Filter data based on observation attribute (for example, pixel quality);

Define the spatio-temporal range of interest independent of data storage unit; and

In [38]:
import datetime
data_request = {'dimensions': {
   'x': {'range': (148.4, 148.5),}, 'y': {'range': (-36.4, -36.5),},
   'time': {'range': (datetime.datetime(2013, 1, 1),
                      datetime.datetime(2013, 12, 30, 23, 59, 59))}}}

Define the specific sensor or combination of sensor data to be analysed.

The API will provide a simplified conceptual model for data query and analysis based
on an n-dimensional array abstraction;

During EDA, lazy evaluation of calculations so only those results that are in use are
computed; and

Support for calculations on arrays that are larger than core memory.

Continental scale product generation will be based off the continental workflows from the
current ADGC v1 API, however it will be modified to use the version 2 data retrieval
mechanism.

The ability to manage results of calculations as a temporary/private data cube for further
analysis.

Web based delivery of products through WMS, WCS, CS/W, OpenDAP services.

Basic provenance that records information about an analysis result/product such as what
datasets, software version, ancillary data and algorithm was used to produce the product.
Wherever possible, Version 2 will adopt and adapt existing software, services and standards.

## Other Requirements
### The Project Plan will be supported by:
 A transition plan and timetable for moving AGDC production from Version 1 to Version 2 of
the AGDC (including controlled updates to data collections that are already accessible via
RDS);

A software release and management plan;

A plan for improving the data management of the collections in the RDS Landsat (rs0) and
WOfs (fk4) Projects, such as establishing data layout, appropriate access controls (including
read/write permissions) and alignment to existing organisational data libraries);
 Reformatting of all data in projects rs0 (Landsat) and fk4 (WOfS) that are currently stored in
GeoTIFF into netCDF4-CF;

An AGDC database API which can provide a base for further AGDC developments and that can
accommodate updates in the internal structure of netCDF4-CF;

Upgrading all documentation (including Data Management Plans, Product Specifications, etc.)
and ensuring that any Metadata is compliant with the requirements of the Australian
Government Spatial Data Policies and Directives, the Australian National Data Service and
data.gov.au; and

 




Benchmarks and Quality Assurance tests to validate the quality of the access that is required
by the agreed use-cases.